In [18]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
from pinecone import Pinecone
pc = Pinecone(api_key=os.getenv('PINECONE_KEY'))
index = pc.Index('default')

In [28]:
import json
from angle_emb import AnglE

business_data_path = 'business_data.json'
with open(business_data_path) as f:
    data = json.load(f)


angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()
emb_vectors = angle.encode(str(data), to_numpy=True)

ids = [str(data['id'])]
meta_data = [data]

to_upsert = zip(ids, emb_vectors, meta_data)
index.upsert(vectors=to_upsert)

{'upserted_count': 1}

In [31]:
user_query = input('Enter your query: ')
query_embeddings = angle.encode(str(data), to_numpy=True).tolist()

In [32]:
result = index.query(
    vector=query_embeddings,
    top_k=5,
    include_values=False,
    include_metadata=True
)

In [34]:
result

{'matches': [{'id': '3',
              'metadata': {'USP': 'Cutting-edge Software Development',
                           'age': '5 years',
                           'company_name': 'PQR Tech Solutions',
                           'employees': 250.0,
                           'id': 3.0,
                           'industry': 'Information Technology',
                           'profit': '$5,000,000',
                           'revenue': '$25,000,000',
                           'sales': '$15,000,000'},
              'score': 0.998860717,
              'values': []},
             {'id': '2',
              'metadata': {'USP': 'Best AI solutions',
                           'age': '2 years',
                           'company_name': 'XYZ Corporation',
                           'employees': 200.0,
                           'id': 2.0,
                           'industry': 'Product Technology',
                           'profit': '$4,000,000',
                           'revenue': '

In [37]:
matched_info = ' '.join(' '.join(str(item['metadata'].get(key, '')) for key in ['USP', 'employees', 'industry', 'sales', 'profit', 'revenue']) for item in result['matches'])
sources = [' '.join(str(item['metadata'].get(key, '')) for key in ['industry', 'USP']) for item in result['matches']]
context = f"Information: {matched_info} and the sources: {sources}"
sys_prompt = f"""
Instructions:
- Be helpful and answer questions concisely. If you don't know the answer, say 'I don't know'
- Utilize the context provided for accurate and specific information.
- Incorporate your preexisting knowledge to enhance the depth and relevance of your response.
- You are answering as a finance and a business guru to people are trying to make their startup successful.
- Help the client with giving advantages and disadvantages glorifying both of them in details and points to help them thrive.
- Cite your sources
Context: {context}
"""

In [38]:
from groq import Groq
client = Groq()

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": f"{sys_prompt}"
        },
        {
            "role": "user",
            "content": f"{user_query}",
        }
    ],
    model="mixtral-8x7b-32768",
    temperature=0.5,
    max_tokens=2048,
    top_p=1,
    stop=None,
    stream=False,
)
print(chat_completion.choices[0].message.content)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


To determine if the business of ABC will be successful, it is important to consider the advantages and disadvantages of the different areas they are investing in.

Advantages of Cutting-edge Software Development:

* Can provide a competitive edge in the market
* Allows for the creation of custom solutions for clients
* Can lead to increased efficiency and productivity
* High demand for software development services

Disadvantages of Cutting-edge Software Development:

* Can be expensive to research and develop new software
* Requires a highly skilled workforce
* Technology can become outdated quickly
* Intense competition in the software development industry

Advantages of Best AI Solutions:

* Can automate repetitive tasks, freeing up employee time
* Can provide valuable insights and predictions through data analysis
* Can improve decision-making and strategic planning
* High demand for AI solutions in various industries

Disadvantages of Best AI Solutions:

* Can be expensive to rese